In [1]:
import pandas as pd

df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df.shape

(20800, 5)

In [3]:
df.isnull().sum()
# No of null values in column

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [4]:
df = df.dropna()
# drop null values

In [5]:
df.shape

(18285, 5)

In [6]:
X = df.drop('label', axis=1)
y = df.label

In [7]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [8]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [9]:
messages = X.copy()
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [10]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


**text pre processing**

In [11]:
import nltk
import re
from nltk.corpus import stopwords

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from nltk.stem.porter import PorterStemmer 

ps = PorterStemmer()

In [ ]:
corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word)
              for word in review 
               if not word in stopwords.words('english')]
    review = ' '.join(review)
    
    corpus.append(review)

In [ ]:
corpus[0]

**onehot representation**

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot

voc_size = 5000
onehot_repr = [one_hot(words, voc_size)
               for words in corpus] 
onehot_repr[0]

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='post',
                              maxlen=sent_length)
embedded_docs[0]

In [ ]:
# Creating model
embedding_vector_features=40 

model=Sequential()

model.add(Embedding(voc_size, embedding_vector_features,
                    input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

In [ ]:
len(embedded_docs), y.shape

In [ ]:
import numpy as np

X_final = np.array(embedded_docs)
y_final = np.array(y)

In [ ]:
X_final.shape

In [ ]:
y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, 
                                                    test_size=0.33, 
                                                    random_state=42)

In [ ]:
# Finally Training
model.fit(X_train, y_train, validation_data=(X_test, y_test),
          epochs=10, batch_size=64)

In [ ]:
from tensorflow.keras.layers import Dropout

## Creating model
embedding_vector_features=40

model=Sequential()

model.add(Embedding(voc_size, embedding_vector_features,
                    input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = np.where(y_pred > 0.6, 1,0)
# AUC ROC Curve

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))